In [16]:
import os
import toml

from dotenv import load_dotenv
load_dotenv()


config = toml.load("config.toml")

In [17]:
import pytubefix
from pytubefix import Playlist, Channel, YouTube

In [18]:
ma = Channel("https://www.youtube.com/@MetricAcademy")

In [19]:
# playlists = ma.playlists
# print(len(playlists))

In [ ]:
# for pl in playlists:
# 	print(pl.title)
    

In [21]:
ma = pytubefix.Channel("https://www.youtube.com/@MetricAcademy")

In [22]:
videos = ma.video_urls

In [ ]:
len(videos)

In [ ]:
from _youtube import YouTubeVideo

video_url = "https://www.youtube.com/watch?v=WIk3iAXErBY"

video = YouTubeVideo(video_url)
tr = video.get_transcript()

# save transcript to file
with open("transcript.srt", "w", encoding="utf-8") as f:
    f.write(tr)

In [ ]:
# video.get_metadata()

In [ ]:
from openai import OpenAI

import os
import toml

from dotenv import load_dotenv
load_dotenv()


config = toml.load("config.toml")

client = OpenAI()


In [ ]:
from pydantic import BaseModel, ValidationError


In [ ]:
import re

class Segment(BaseModel):
    title: str
    start_time: str  # e.g. '00:03:15'

    @classmethod
    def __get_validators__(cls):
        yield from super().__get_validators__()
        yield cls.validate_time_format

    @staticmethod
    def validate_time_format(value):
        if not re.match(r"^\d{2}:\d{2}:\d{2}$", value):
            raise ValueError("start_time must be in HH:MM:SS format")
        return value
 
class TimeStamps(BaseModel):
    segments: list[Segment]

In [ ]:
prompt = """You are given a YouTube transcript with timestamps. Identify 3-7 key segments and return \ntitle and start_time\n\n[INSTRUCTIONS]The text should be in Armenian, you are allowed to use English for terminology or technical terms if needed.\n\nYou need to be technical and precise. Make sure that the segments cover the entire video and are spaced with at least 5-10 minutes apart. Keep the format HH:MM:SS for timestamps. An example return is 00:00 Tuple
14:00 Set
31:54 Dictionary մոտիվացիա
40:39 Dictionary"""

In [ ]:
def generate_segmented_timestamps(transcript: str) -> str:
    """
    Uses the OpenAI chat endpoint (api-mode=chat) to analyze a transcript
    and return key segment titles with start timestamps as JSON.
    """
    # Create chat completion
    print(prompt)
    response = client.chat.completions.parse(
        model=config["openai"]["model"],
        messages=[
            {
                "role": "system",
                "content": prompt
            },
            {
                "role": "user",
                "content": transcript
            }
        ],
        response_format=TimeStamps,
        
    )

    # Extract and return the assistant's content
    return response.choices[0].message.parsed


In [ ]:
with open("transcript.srt", "r") as f:
    transcript_text = f.read()

In [ ]:
res = generate_segmented_timestamps(transcript_text)

You are given a YouTube transcript with timestamps. Identify 3-7 key segments and return 
title and start_time

[INSTRUCTIONS]The text should be in Armenian, you are allowed to use English for terminology or technical terms if needed.

You need to be technical and precise. Make sure that the segments cover the entire video and are spaced with at least 5-10 minutes apart. Keep the format HH:MM:SS for timestamps. An example return is 00:00 Tuple
14:00 Set
31:54 Dictionary մոտիվացիա
40:39 Dictionary


In [ ]:
for segment in res.segments:
    print(f"{segment.start_time} {segment.title}")


00:00:00 Ներածություն և պլան
00:14:00 Python գրադարաններ եւ գործիքներ
00:34:00 YouTube տվյալներ մշակելն ու ստանալը
01:00:00 Աուդիո և վիդեո ներբեռնում
01:14:00 Թարգմանական գործիքներ: Google Translate եւ Deepl
01:34:00 Նախապատրաստական եւ ամփոփում


# Antrophic

In [ ]:
from _youtube import YouTubeVideo

video_url = input("Enter YouTube video URL: ")

video = YouTubeVideo(video_url)
tr = video.get_transcript()

# save transcript to file
with open("transcript.srt", "w", encoding="utf-8") as f:
    f.write(tr)

import re

# ── configurable filters ────────────────────────────────────────────────
TIMESTAMP_RGX      = re.compile(r'^\d{2}:\d{2}:\d{2},\d{3}\s*-->', re.M)   # 00:00:00,000 -->
STAGE_DIR_RGX      = re.compile(r'^[\[(].*?[\])]$', re.M)                  # [Music], (laughs)
CUE_INDEX_RGX      = re.compile(r'^\d+\s*$', re.M)                         # 742
BLANK_LINE_RGX     = re.compile(r'^\s*$', re.M)                            # empty lines
# ────────────────────────────────────────────────────────────────────────

def clean_srt_text(srt_text: str, join_paragraphs: bool = False) -> str:
    """
    Strip cue numbers, timestamps, blank lines and bracketed
    stage directions from a YouTube­-generated SRT transcript.

    Parameters
    ----------
    srt_text : str
        Raw transcript exactly as read from an .srt file.
    join_paragraphs : bool, default=False
        If True, collapses all remaining lines into a single paragraph.

    Returns
    -------
    str
        Cleaned transcript text.
    """
    # Remove each class of “extra stuff”
    txt = TIMESTAMP_RGX.sub('', srt_text)
    txt = CUE_INDEX_RGX.sub('', txt)
    txt = STAGE_DIR_RGX.sub('', txt)
    txt = BLANK_LINE_RGX.sub('\n', txt)     # normalise multiple blank lines

    # Strip leading/trailing whitespace on each line
    lines = [ln.strip() for ln in txt.splitlines() if ln.strip()]
    lines = [lines[i] for i in range(1, len(lines), 4)]
    # take every 4th line (the actual text)
 

    return (' '.join(lines) if join_paragraphs else '\n'.join(lines))

# ── example usage ────────────────────────────────────────────────────────
if __name__ == "__main__":
    # imagine srt_text came from an HTTP request, clipboard, etc.
    with open("transcript.srt", encoding="utf-8") as f:
        srt_text = f.read()

    tr_clean = clean_srt_text(srt_text)              # multi-line output
    # print(clean_srt_text(srt_text, True))      # one long paragraph
tr_clean

'Ողջույն, էսօր տենց երեք կտորից ա դասը\nոր առաջին անգամ Python-ից դուրս կգանք,\nմինիմալոտ մակարդակի էն ամենա հիմնական\nկբռնի մի հատ ուրիշ բանի հետ, որ\nէն ա, որ\nենք\nբաներ են, որ ծրագրավորողը պետք է իմանադ\nֆայլերի հետ աշխատել ինչ-որ ֆայլեր բացել\nերրորդ բանն էլ այն որ կսովորենք մի քանի\nգրադարանների պրինցիպը աշխատելու որն ա ու\nներմուծել ու էդ տենց մի աշխարհ\nու մի հատ հետաքրքիր գրադարան կա որ\nօգտագործել, չենք ուսումնասիրել\nսահմանափակենք, ձեռով գրենք էն ինչ պետք\nայսօրվանից հետո արդեն մի քիչ այդ\nդուրս\nանցնենք կախեցզվեց\nկապված նախ ոնց բացենք ընդհանրապես\nտերմինալը տերմինալը կարանք սահմանենք\nաշխատացնելու գործիքս ասած ու օրինակի\nտեղ մի վայրկյան\nհատ թազա ֆայլ ֆայլի անունը փոխենք, մի\nայլն, և այլն, հեսա կտեսնենք ոնց կարանք\nինչը առաջին հերթին կարի բավականին\nերկրորդը կարի անհրաժեշտ ուղղակի, երբ որ\nորտեղ մենք ուղղակի էկրան չենք ունենալու,\nօրինակ, գանք, սպեն ինչ ուղղակի\nաշխատացվի։ Ըը ու հա, տերմինալ բացելու\nտերմինալ կամ cmd կամ կոնսոլ տենց տարբեր\nբան, որ եթե գաք ուղղակի ստեղ այ

: 

In [ ]:



prompt = """You are given a YouTube transcript with timestamps. Identify 3-7 key segments and return \ntitle and start_time in the format [start_time] [title]\n\n[INSTRUCTIONS]The text should be in Armenian, you are allowed to use English for terminology or technical terms if needed.\n\nYou need to be technical and precise. Make sure that the segments cover the entire video and are spaced with at least 5-10 minutes apart. Keep the format HH:MM:SS for timestamps. An example return is 00:00 Tuple
14:00 Set
31:54 Dictionary մոտիվացիա
40:39 Dictionary. Return only a text with the segments in the format:
# 00:00 Segment Title ..."""


message = client.messages.create(
    # model="claude-opus-4-20250514",
    model="claude-3-5-sonnet-20241022",
    max_tokens=1024,
    system=prompt,
    messages=[
        {"role": "user", "content": tr_clean}
    ]
)

text = message.content[0].text


In [ ]:

print("Վարկյանները գեներացված են ԱԲ-ի կողմից, ձեռքով նորմալ չեմ ուղղել")
print(text.replace("#", ""))

Վարկյանները գեներացված են ԱԲ-ի կողմից, ձեռքով նորմալ չեմ ուղղել
Here are the key segments from the video lecture:

00:00:00 Գործնական խնդիր - Password գեներատոր
- Առաջադրանք՝ ստեղծել password գեներատոր ֆունկցիա 
- Տարբեր բարդության մակարդակներ (easy, medium, hard)
- Թվեր, տառեր և սիմվոլներ օգտագործելով

00:29:30 Random մոդուլի կիրառում
- Random մոդուլի հիմնական ֆունկցիաներ
- Random choice և uniform ֆունկցիաներ
- Պատահական թվերի գեներացում

00:47:00 Մեծ թվերի օրենքի օրինակ
- Կոպեկի գցման սիմուլյացիա 
- Հավանականությունների հաշվարկ
- Պատահականության վիզուալիզացիա գրաֆիկով

01:15:00 Matplotlib գրաֆիկների օգտագործում
- Pyplot մոդուլի import
- Գրաֆիկների կառուցում
- Տվյալների վիզուալիզացիա

01:30:00 Դասի ամփոփում և հաջորդ թեմաներ
- Գրադարանների օգտագործման նախապատրաստում
- Python-ի հնարավորությունների ընդլայնում
- Հետագա ուսուցման պլաններ


In [ ]:

# # extract ```json part
# json_part = text.split("```json")[1].split("```")[0].strip()
# import json
# res = json.loads(json_part)
# for segment in res:
# 	print(f"{segment['start_time']} {segment['title']}")

In [ ]:
# res = eval(message.content[0].text.replace("\n", " ").replace("  ", " ")[8:-4])

In [ ]:
# res

In [ ]:
print("Վարկյանները գեներացված են ԱԲ-ի կողմից, ձեռքով նորմալ չեմ ուղղել")
for segment in res:
    print(f"{segment['start_time']} {segment['title']}")


Վարկյանները գեներացված են ԱԲ-ի կողմից, ձեռքով նորմալ չեմ ուղղել
00:00:00 Գործնական դասերի ավարտ և հաջորդ թեմաների պլան
00:11:00 YouTube-ից վիդեո մետադատաների քաշում dataclass-ով
00:27:00 Transcript (ենթագրերի) քաշում և աշխատանք
00:33:00 Աուդիո և վիդեո ֆայլերի ներբեռնում YouTube-ից
00:52:00 Google Translate և DeepL API-ների հետ աշխատանք
01:13:00 Աբստրակտ կլասներ և թարգմանիչների Pipeline
01:26:00 Git և քոմիթների մասին քննարկում


In [ ]:
import anthropic

client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=os.getenv("CLAUDE_API_KEY"),
)

# Playlist

In [6]:
import re
import pandas as pd
from tqdm import tqdm
from _youtube import YouTubeVideo
from pytubefix import Playlist, YouTube, Channel

In [7]:
ma = Channel("https://www.youtube.com/@MetricAcademy")

In [8]:
ma.playlists

[<pytubefix.contrib.Playlist object: playlistId=PLfLD2TpGxVUxu2_l1zR1RpGQqT9fp-WIR>, <pytubefix.contrib.Playlist object: playlistId=PLfLD2TpGxVUytC30DnVEKt-inCmsylOpY>, <pytubefix.contrib.Playlist object: playlistId=PLfLD2TpGxVUxfiN8AF5b-LFDvSrT_dVj6>, <pytubefix.contrib.Playlist object: playlistId=PLfLD2TpGxVUzpXv7Bey_8qkVKPSMhN2cS>, <pytubefix.contrib.Playlist object: playlistId=PLfLD2TpGxVUwro_kavF8BvvJXmyXB7jVF>, <pytubefix.contrib.Playlist object: playlistId=PLfLD2TpGxVUwUa_o8Fxig8SlC1PrLYWUg>, <pytubefix.contrib.Playlist object: playlistId=PLfLD2TpGxVUxjP7WnHxAWfbd8LWSY-pix>, <pytubefix.contrib.Playlist object: playlistId=PLfLD2TpGxVUwbO3Ts-JzkLTRaFcpTBgNL>, <pytubefix.contrib.Playlist object: playlistId=PLfLD2TpGxVUwkKFfZYCeIip7BHzmk0mbK>, <pytubefix.contrib.Playlist object: playlistId=PLfLD2TpGxVUwNhdJWh3lpAJLFBjV6ilqv>]

In [9]:
len(ma.video_urls)

10

In [ ]:
for pl in tqdm(ma.playlists):
    pl_title = pl.title
    print(pl_title)
    pl_title_no_special_chars = re.sub(r'[^\w\s]', '', pl_title)
    print(pl_title_no_special_chars)
    
    res = []

    for v in tqdm(pl.video_urls):
        video = YouTubeVideo(v)
        
        data = video.get_metadata()
        res.append(data)
    
    res_df = pd.DataFrame(res)
    res_df.to_csv(f"out/{pl_title_no_special_chars}.csv", index=False)


  0%|          | 0/10 [00:00<?, ?it/s]

Python Libraries
Python Libraries


 10%|█         | 1/10 [00:06<00:55,  6.19s/it]

Tuple-ներ և Set-եր | 24 բաժին խնդիրներ | Python Profound
Tupleներ և Setեր  24 բաժին խնդիրներ  Python Profound


 20%|██        | 2/10 [00:28<02:07, 15.91s/it]

...

Չմոնտաժված տեսագրություններ
Չմոնտաժված տեսագրություններ


 90%|█████████ | 9/10 [07:39<00:50, 51.00s/it]

Python - Դասեր
Python  Դասեր


100%|██████████| 10/10 [09:40<00:00, 58.03s/it]


AttributeError: 'Channel' object has no attribute 'playlist_'

In [12]:

with open("out/export_info.txt", "w", encoding="utf-8") as f:
    f.write(f"Exported {len(ma.playlists)} playlists with {len(ma.video_urls)} videos.\n")
    f.write(f"Date: {pd.Timestamp.now()}\n")

In [13]:
# pandas merge all cvs on a folder into one excel file
import os
import pandas as pd

folder_path = "out"
all_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
with pd.ExcelWriter("out/merged_data.xlsx") as writer:
    for file in all_files:
        df = pd.read_csv(os.path.join(folder_path, file))
        sheet_name = os.path.splitext(file)[0][:31]  # Excel sheet name max length is 31
        df.to_excel(writer, sheet_name=sheet_name, index=False)

# Armenian transcript

In [13]:
vid = YouTubeVideo("https://www.youtube.com/watch?v=3MqYE2UuN24")

tr = vid.get_transcript("en", return_srt=False)
print(f"Num characters {len(tr)} | Num words {len(tr.split())} | Num sentences {len(tr.split('.'))}")


Num characters 2271 | Num words 431 | Num sentences 16


In [14]:
from googletrans import Translator

async def google_translate(text, target_lang):
    async with Translator() as translator:
        result = await translator.translate(text, dest=target_lang)
        return result.text

In [15]:
translation = await google_translate(tr, "hy")

In [16]:
translation

'Այդ ցուրտ, անձրեւոտ օրերին, երբ մոռանում ես անձրեւի բաճկոնը կամ հովանոցը, եւ ուզում ես հնարավորինս չոր մնալ ... պետք է քայլես եւ ավելի շատ ժամանակ անցկացիր անձրեւի տակ: Կամ պետք է գործարկեք, ինչը նշանակում է, որ դուք կխորտակվեք ավելի շատ անձրեւաջրերի կողքից: Ենթադրելով, որ դուք դեռ լիովին ներծծված չեք եղել, եւ դուք չեք նետվում պուդլների մեջ, պատասխանը պարզ է: Երբ դուրս եք գալիս անձրեւաջրերի մեկ ճանապարհից, դուք անցնում եք մեկ ուրիշի ճանապարհով: Այսպիսով, ձեր գագաթին հարվածող անձրեւի քանակը կայուն է, անկախ այն բանից, թե որքան արագ եք գնում: Այլապես, կարող եք պատկերացնել, որ անձրեւներն իրենք են գրենական պիտույքներ, եւ դուք (եւ ձեր տակ գտնվող երկիրը) շարժվում են անձրեւի միջով: Եվ քանի որ զուգահեռացված (դա 3D զուգահեռ) ծավալը կախված չէ իր թեքությունից, այնուհետեւ անկախ նրանից, թե որքան արագ եք հորիզոնական շարժվում, նույն քանակությամբ անձրեւի վրա կուղեւորվի ձեզանից յուրաքանչյուր վայրկյան: Հիմա, եթե դուք չեք շարժվում, վերեւից անձրեւը բոլորը կստանաք: Բայց եթե շարժվում ես, կողքից նույնպես վազ

In [38]:
# !pip install --upgrade google-api-python-client
!pip install google-genai

  Using cached google_genai-1.23.0-py3-none-any.whl.metadata (38 kB)
  Using cached tenacity-8.5.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached websockets-15.0.1-cp310-cp310-win_amd64.whl.metadata (7.0 kB)
Using cached google_genai-1.23.0-py3-none-any.whl (223 kB)
Using cached tenacity-8.5.0-py3-none-any.whl (28 kB)
Using cached websockets-15.0.1-cp310-cp310-win_amd64.whl (176 kB)

   ---------------------------------------- 0/3 [websockets]
   ---------------------------------------- 0/3 [websockets]
   ---------------------------------------- 0/3 [websockets]
   ---------------------------------------- 0/3 [websockets]
   ---------------------------------------- 0/3 [websockets]
   ---------------------------------------- 0/3 [websockets]
   ---------------------------------------- 0/3 [websockets]
   ------------- -------------------------- 1/3 [tenacity]
   -------------------------- ------------- 2/3 [google-genai]
   -------------------------- ------------- 2/3 [google-genai

In [43]:
from dotenv import load_dotenv
load_dotenv()

True

In [25]:
text = """Այն ցուրտ, անձրևոտ օրերին, երբ մոռանում եք ձեր անձրևանոցը և ցանկանում եք հնարավորինս չոր մնալ ... պետք է քայլե՞լ և ավելի շատ ժամանակ անցկացնել անձրևի տակ: Թե՞ պետք է վազել, ինչը նշանակում է, որ կբախվենք նաև կողքից ընկնող անձրևի կաթիլների հետ: 
...
Ենթադրելով, որ դուք դեռ ամբողջությամբ չեք թրջվել և չեք թռչում ջրակույտերի մեջ, պատասխանը պարզ է: Երբ դուք շարժվում եք մի ընկնող անձրևի կաթիլից դուրս, դուք շարժվում եք մեկ այլ կաթիլի ուղղությամբ: Այսպիսով, ձեզ վերևից հարվածող անձրևի կաթիլների քանակը հաստատուն է՝ անկախ նրանից, թե որքան արագ եք շարժվում: 
...
Որպես այլընտրանք՝ կարող եք պատկերացնել, որ անձրևօ կաթիլները անշարքժ են, իսկ դուք (ձեր տակ գտնվող երկրագնդի հետ միասին) շարժվում ենք վերև անձրևի միջով: 
...
Եվ քանի որ զուգահեռանիստի (նույն ինքը՝ երեք դ զուգահեռագծի) ծավալը կախված չէ նրա թեքությունից, ապա անկախ նրանից, թե որքան արագ եք հորիզոնական շարժվում, ամեն վարկյան ձեր վրա ընկած անձևի քանակը նույնն է լինելու։
...
Եթե դուք չեք շարժվում, ապա վերևից ընկնող անձրևի կաթիլները միակն ենք որոնց կբախվենք: Բայց եթե շարժվում ենք, ապա նաև կբախվենք կողքից եկող անձրևի կաթլիներին և ավելի շատ կթրջվենք: Այսպիսով, ամեն վայրկյան, ամենաչորը մնալու համար պետք է անշարժ կանգնենք, և որքան ավելի արագ շարժվենք, այնքան ավելի շատ կթրջվենք:
...
Բայց եթե փորձում եք հասնել A կետից B կետ, ապա անշարժ կանգնելը լավագույն միտքը չէ: Եվ A կետից B կետ գնալու ժամանակ, անձրևի ընդհանուր քանակը, որին կբախվեք կողքից, կախված չէ նրանից, թե որքան արագ եք գնում - ճիշտ այնպես, ինչպես ձյունամաքրող մեքենան նույն քանակությամբ ձյուն կմաքրի ճանապարհի մի հատվածից՝ անկախ նրա արագությունից: Անձրևի միջով վազելու դեպքի համար կարող եք դա պարզել կրկին զուգահեռանիստների օգնությամբ:
...
Այսպիսով, ձեր ընդհանուր թրջվածությունը հավասար է ամեն վարկյան վերևից եկող անձրևի պատճառով թրջվածությանը՝ բազմապատկած անձրևի տակ անցկացրած ժամանակի քանակով, գումարած կողքից եկող անձրևի թրջվածությանը ամեն մետրի համար՝ բազմապատկած ձեր անցած մետրերի քանակով:
...
Հիշենք որ ֆիքսված ժամանակահատվածում նույն քանակությամբ անձրև կհարվածի ձեզ վերևից՝ անկախ նրանից, թե որքան արագ եք գնում: Եվ տվյալ հեռավորության վրա՝ նույն քանակությամբ անձրև կհարվածի կողքից - կրկին, անկախ նրանից, թե որքան արագ եք գնում:
...
Իմի բերելով, մի կետից մյուսը գնալիս ամենաչորը մնալու համար, պետք է փորձել նվազագույնի հասցնել վերևից ձեր վրա ընկնող ջրի քանակը: Եվ բավականին պարզ է, որ դա նշանակում է անձրևից հնարավորինս արագ փախչել:
"""

In [26]:
tr = text
print(f"Num characters {len(tr)} | Num words {len(tr.split())} | Num sentences {len(tr.split('.'))}")


Num characters 2326 | Num words 354 | Num sentences 4


In [27]:
# To run this code you need to install the following dependencies:
# pip install google-genai

import base64
import mimetypes
import os
import re
import struct
from google import genai
from google.genai import types


def save_binary_file(file_name, data):
    f = open(file_name, "wb")
    f.write(data)
    f.close()
    print(f"File saved to to: {file_name}")


def generate():
    client = genai.Client(
        api_key=os.environ.get("GEMINI_API_KEY"),
    )

    model = "gemini-2.5-pro-preview-tts"
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text=text),
            ],
        ),
    ]
    generate_content_config = types.GenerateContentConfig(
        temperature=1,
        response_modalities=[
            "audio",
        ],
        speech_config=types.SpeechConfig(
            voice_config=types.VoiceConfig(
                prebuilt_voice_config=types.PrebuiltVoiceConfig(
                    voice_name="Zephyr"
                )
            )
        ),
    )

    file_index = 0
    for chunk in client.models.generate_content_stream(
        model=model,
        contents=contents,
        config=generate_content_config,
    ):
        if (
            chunk.candidates is None
            or chunk.candidates[0].content is None
            or chunk.candidates[0].content.parts is None
        ):
            continue
        if chunk.candidates[0].content.parts[0].inline_data and chunk.candidates[0].content.parts[0].inline_data.data:
            file_name = f"first_attmempt_{file_index}"
            file_index += 1
            inline_data = chunk.candidates[0].content.parts[0].inline_data
            data_buffer = inline_data.data
            file_extension = mimetypes.guess_extension(inline_data.mime_type)
            if file_extension is None:
                file_extension = ".wav"
                data_buffer = convert_to_wav(inline_data.data, inline_data.mime_type)
            save_binary_file(f"{file_name}{file_extension}", data_buffer)
        else:
            print(chunk.text)

def convert_to_wav(audio_data: bytes, mime_type: str) -> bytes:
    """Generates a WAV file header for the given audio data and parameters.

    Args:
        audio_data: The raw audio data as a bytes object.
        mime_type: Mime type of the audio data.

    Returns:
        A bytes object representing the WAV file header.
    """
    parameters = parse_audio_mime_type(mime_type)
    bits_per_sample = parameters["bits_per_sample"]
    sample_rate = parameters["rate"]
    num_channels = 1
    data_size = len(audio_data)
    bytes_per_sample = bits_per_sample // 8
    block_align = num_channels * bytes_per_sample
    byte_rate = sample_rate * block_align
    chunk_size = 36 + data_size  # 36 bytes for header fields before data chunk size

    # http://soundfile.sapp.org/doc/WaveFormat/

    header = struct.pack(
        "<4sI4s4sIHHIIHH4sI",
        b"RIFF",          # ChunkID
        chunk_size,       # ChunkSize (total file size - 8 bytes)
        b"WAVE",          # Format
        b"fmt ",          # Subchunk1ID
        16,               # Subchunk1Size (16 for PCM)
        1,                # AudioFormat (1 for PCM)
        num_channels,     # NumChannels
        sample_rate,      # SampleRate
        byte_rate,        # ByteRate
        block_align,      # BlockAlign
        bits_per_sample,  # BitsPerSample
        b"data",          # Subchunk2ID
        data_size         # Subchunk2Size (size of audio data)
    )
    return header + audio_data

def parse_audio_mime_type(mime_type: str) -> dict[str, int | None]:
    """Parses bits per sample and rate from an audio MIME type string.

    Assumes bits per sample is encoded like "L16" and rate as "rate=xxxxx".

    Args:
        mime_type: The audio MIME type string (e.g., "audio/L16;rate=24000").

    Returns:
        A dictionary with "bits_per_sample" and "rate" keys. Values will be
        integers if found, otherwise None.
    """
    bits_per_sample = 16
    rate = 24000

    # Extract rate from parameters
    parts = mime_type.split(";")
    for param in parts: # Skip the main type part
        param = param.strip()
        if param.lower().startswith("rate="):
            try:
                rate_str = param.split("=", 1)[1]
                rate = int(rate_str)
            except (ValueError, IndexError):
                # Handle cases like "rate=" with no value or non-integer value
                pass # Keep rate as default
        elif param.startswith("audio/L"):
            try:
                bits_per_sample = int(param.split("L", 1)[1])
            except (ValueError, IndexError):
                pass # Keep bits_per_sample as default if conversion fails

    return {"bits_per_sample": bits_per_sample, "rate": rate}


if __name__ == "__main__":
    generate()


File saved to to: first_attmempt_0.wav


In [46]:
yt

NameError: name 'yt' is not defined

# Audio video download

In [4]:
from _youtube import YouTubeVideo


In [7]:
yt = YouTubeVideo("https://www.youtube.com/watch?v=3MqYE2UuN24")

yt.download_audio(output_path="running_in_the_rain")

In [8]:
yt.download_video(output_path="running_in_the_rain", highest_quality=True)

In [9]:
yt.download_transcript(output_path="running_in_the_rain", language="en")

In [17]:
transcript_text = yt.get_transcript("en", return_srt=False)


In [19]:
translation

'Այդ ցուրտ, անձրեւոտ օրերին, երբ մոռանում ես անձրեւի բաճկոնը կամ հովանոցը, եւ ուզում ես հնարավորինս չոր մնալ ... պետք է քայլես եւ ավելի շատ ժամանակ անցկացիր անձրեւի տակ: Կամ պետք է գործարկեք, ինչը նշանակում է, որ դուք կխորտակվեք ավելի շատ անձրեւաջրերի կողքից: Ենթադրելով, որ դուք դեռ լիովին ներծծված չեք եղել, եւ դուք չեք նետվում պուդլների մեջ, պատասխանը պարզ է: Երբ դուրս եք գալիս անձրեւաջրերի մեկ ճանապարհից, դուք անցնում եք մեկ ուրիշի ճանապարհով: Այսպիսով, ձեր գագաթին հարվածող անձրեւի քանակը կայուն է, անկախ այն բանից, թե որքան արագ եք գնում: Այլապես, կարող եք պատկերացնել, որ անձրեւներն իրենք են գրենական պիտույքներ, եւ դուք (եւ ձեր տակ գտնվող երկիրը) շարժվում են անձրեւի միջով: Եվ քանի որ զուգահեռացված (դա 3D զուգահեռ) ծավալը կախված չէ իր թեքությունից, այնուհետեւ անկախ նրանից, թե որքան արագ եք հորիզոնական շարժվում, նույն քանակությամբ անձրեւի վրա կուղեւորվի ձեզանից յուրաքանչյուր վայրկյան: Հիմա, եթե դուք չեք շարժվում, վերեւից անձրեւը բոլորը կստանաք: Բայց եթե շարժվում ես, կողքից նույնպես վազ

In [22]:
import pandas as pd

pd.DataFrame({"english": [transcript_text],
              "translation": [translation]}).to_csv("running_in_the_rain/text.csv")

# Auto dubbing